In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from pandas import DataFrame

In [ ]:
term7=pd.read_csv("/kaggle/input/instacart-term7-term30/term7.csv")
term30=pd.read_csv("/kaggle/input/instacart-term7-term30/term30.csv")

**Word2Vec for term7**

In [ ]:
# term7 = term30
# 이번엔 30에 대해서임

In [ ]:
term7.head()

In [ ]:
# len(term7) #3586305

In [ ]:
# len(term30) #3477322

In [ ]:
from collections import Counter as cc

pro_name = cc(term7['product_name'])
A = pro_name.most_common()[:10000]

In [ ]:
A = DataFrame(A)
A.columns = ["product_name","product_count"]

In [ ]:
A.head()

In [ ]:
# print(A)

In [ ]:
top7 = A.join(term7.set_index('product_name'), on='product_name')

In [ ]:
len(top7)

In [ ]:
# print(top7)

make dataset

In [ ]:
top7['user_order'] =top7['user_id'] + top7['order_number'] * 0.1

In [ ]:
top7.drop(['eval_set', 'product_count', 'product_id', 'user_id', 'order_number'], axis='columns', inplace=True)
top7.head()

In [ ]:
group7 = top7["product_name"].groupby(top7['user_order'])
group7 = pd.DataFrame(group7)

위 코드로 user_order와 user_order기준으로 groupby된 product_list가 순서대로 자동으로 열로 추가된 group7이 만들어지고<br>
열 이름은 개념적인 것이기 때문에, 지금은 0 1 로 인덱스만 적혀있어서 .columns로 열 이름을 지정해주는것

In [ ]:
group7.columns = ['user_order','product_list']

In [ ]:
group7.head()

In [ ]:
product = [list(s) for s in group7.product_list]

In [ ]:
type(group7.user_order)

In [ ]:
print(group7.product_list[0])

In [ ]:
# product

In [ ]:
len(group7)

![](http://)Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
%%time
model = Word2Vec(product)

In [ ]:
model.init_sims(replace=True)


0	Banana	59420<br>
1	Bag of Organic Bananas	44119<br>
2	Organic Strawberries	29586<br>
3	Organic Baby Spinach	27177<br>
4	Organic Hass Avocado	23841<br>
5	Organic Avocado	20513<br>
6	Large Lemon	17851<br>
7	Strawberries	16851<br>
8	Organic Whole Milk	15885<br>
9	Limes	15765<br>

In [ ]:
model.wv.similarity('Banana', 'Bag of Organic Bananas')

In [ ]:
model.wv.most_similar("Organic Rolled Oats")

In [ ]:
model.wv.most_similar("Tart Cherry Yoghurt")

In [ ]:
model.wv.most_similar("Yogurt, Strained Low-Fat, Coconut")

In [ ]:
model.wv.most_similar("Organic Strawberries")

In [ ]:
model.wv.most_similar("Organic Baby Spinach")

In [ ]:
model.wv.most_similar("Organic Hass Avocado")

In [ ]:
model.wv.most_similar("Organic Avocado")

In [ ]:
model.wv.most_similar("Large Lemon")

In [ ]:
model.wv.most_similar("Strawberries")

In [ ]:
model.wv.most_similar("Organic Whole Milk")

In [ ]:
model.wv.most_similar("Limes")

Plot

low_dim_embs 내부 갯수와 n_components가 같아야 한다.

n_components : 차원. default는 2.

perplexity : 가장 가까운 이웃 갯수. 보통 5~50. default는 30.

n_iter : 최적화에 사용할 반복 횟수. 최소 200. default는 1000.

init : embedding 초기화 방법. random과 pca 중에서 선택. pca가 보다 안정적. default는 random.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(128, 128)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
# tsne_plot(model)

In [ ]:
# model = Word2Vec(product, size=100, window=5, min_count=1, workers=4)
# model.save('term7_top10000.model')

> **K-Means**

https://ai.intelligentonlinetools.com/ml/k-means-clustering-example-word2vec/

getting Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(product, min_count=1)

In [ ]:
print(model.similarity('Banana', 'Bag of Organic Bananas'))

In [ ]:
print(model.most_similar(positive=['Banana'], negative=[], topn=2))

In [ ]:
# print(list(model.wv.vocab))
# print(len(list(model.wv.vocab)))

k means clustering with NLTK Library

In [ ]:
X = model[model.wv.vocab]

'Word2Vec' object has no attribute 'vocab'
<br> .wv필요

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
NUM_CLUSTERS=3
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
# print(assigned_clusters)
# output = [2, 2, 2, 2, .... 1, 0, 1, 0]

In [ ]:
words = list(model.wv.vocab)
# for i, word in enumerate(words):
#     print (word + ":" + str(assigned_clusters[i]))

In [ ]:
# output =
# Organic Avocado:2
# Organic Whole String Cheese:2
# Granny Smith Apples:2
# Green Beans:2
# .....

In [ ]:
df = pd.DataFrame()

In [ ]:
df['product']=words

In [ ]:
df.head()

In [ ]:
df = df.assign(labels=assigned_clusters)

In [ ]:
df.to_csv("kmeans_7_10000.csv", index=False)

In [ ]:
df.head()

In [ ]:
k7_0 = df[df['labels'] == 0]

In [ ]:
k7_1 = df[df['labels'] == 1]

In [ ]:
k7_2 = df[df['labels'] == 2]

In [ ]:
# k7_3 = df[df['labels'] == 3]

In [ ]:
# k7_4 = df[df['labels'] == 4]

In [ ]:
print(len(k7_0),len(k7_1),len(k7_2))

0 1 2<br>
3680 4743 1577<br>
<br>
0 1 2 3 <br>
3492 4244 1331 933<br>
<br>
0 1 2 3 4<br>
3433 3859 1122 851 735

visualize k-means

https://lovit.github.io/nlp/representation/2018/09/28/tsne/

**DBSCAN**

In [ ]:
model = Word2Vec(sentences = product, size = 100, window = 5, min_count = 500, workers = 4, sg = 1)  # skip_gram 방식
# user_id 분석은 그대로 했고 이번에는 파라미터도 그대로 들고와보기

In [ ]:
from sklearn.cluster import DBSCAN
from gensim.models import Word2Vec

import pandas as pd
import re

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


In [ ]:
# 워드 벡터를 클러스터링하기 위해서
word_vector = model.wv.vectors
word_vector
match_index = model.wv.index2word
model.init_sims(replace=True)


In [ ]:
dbscan = DBSCAN(eps=0.75, min_samples=4)
clusters = dbscan.fit_predict(word_vector) # 워드 벡터를 클러스터링 # fit_predict 함수는 클러스터링 된 결과를 리스트로 산출해준다. 

In [ ]:
df = pd.DataFrame(clusters, columns=["cluster"], index=match_index).reset_index()
df.columns = ["word", "cluster"]
print(df.head())

In [ ]:
# 노이즈 포인트 제거
df = df[df["cluster"] != -1] #-1은 노이즈 포인트

In [ ]:
print(df.groupby(["cluster"]).count())

In [ ]:
min_cluster = df["cluster"].min()
max_cluster = df["cluster"].max()

In [ ]:
print(min_cluster, max_cluster)

In [ ]:
for df_num in range(min_cluster, max_cluster + 1):
    df_index = df[df["cluster"] == df_num].index
    df.loc[df_index, "value"] = list(range(0, len(df_index) * 3, 3))

In [ ]:
df["cluster"].nunique()

In [ ]:
# -- 데이터프레임 출력 전체폭을 1000자로 확장
pd.set_option('display.width', 1000)

# -- 데이터프레임 출력 전체행을 1000개로 확장
# pd.set_option('display.height', 1000)

# -- 데이터프레임 컬럼 길이 출력 제약을 제거
pd.set_option('display.max_colwidth', -1)
 
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', 500)

In [ ]:
grouped = df["word"].groupby(df["cluster"])
grouped = pd.DataFrame(grouped)
grouped.columns = ['cluster','word']
grouped

In [ ]:
font = fm.FontProperties(size=70)
fig, ax = plt.subplots(figsize=(300, 225))
df.plot.scatter(x="cluster", y="value", ax=ax)
df[["cluster", "value", "word"]].apply(lambda x: ax.text(*x, fontproperties=font), axis=1)
plt.show()

In [ ]:
grouped.to_csv("term7_4_10000_dbscan.csv", index=False)